# CHIP/311/Open Baltimore Mismatch

In [118]:
import pandas as pd
from sodapy import Socrata
import pyodbc
conn1=pyodbc.connect('Driver={SQL Server};'
                      'Server=balt-sql-fc;'
                      'Database=CitiStat;'
                      'Trusted_Connection=yes;')
conn2=pyodbc.connect('Driver={SQL Server};'
                      'Server=balt-gslistener;'
                      'Database=housing;'
                      'UID=egis_reader;PWD=baltimore01')

### Open Baltimore Pull

In [ ]:
client = Socrata("data.baltimorecity.gov",None)

query = """
select servicerequestnum,srstatus,createddate
where 1=1
    and SRType = 'SW-HGW'
    and srstatus not like '%Duplicate%'
    and ((srstatus = 'Closed' and statusdate > '2020-02-12') or (srstatus != 'Closed'))
    and createddate < '2020-02-12'
limit 5000
"""

results = client.get("9agw-sxsr",query= query)

# Convert to pandas DataFrame
ob = pd.DataFrame.from_records(results)

### Citistat CSR Pull

In [127]:
csr = pd.read_sql('''select [Service Request Number] as srid, [Created Date], [Block_Number_c],[sr status]
from CitiStat.dbo.CSR
where 1=1
and agency = 'Solid Waste'
and [SR Type] = 'SW-HGW'
and lower([SR Status]) not like '%duplicate%'
and [Created Date] < '2020-02-12'
and (([Status Date] >='2020-02-12' and [SR Status] = 'Close') or ([SR Status] not like '%Close%'))
''',conn1)

### Housing CHIP pull (matches Citistat CHIP tables)

#### This table was used to link CHIP and SR

In [128]:
chip = pd.read_sql('''
select CB_ID, CSR_Num, DateCreate,Block,Lot,status
from housing.dbo.tbl_CHIP_WorkOrder
where 1=1
and [WorkOrderType] = 'Cleaning'
and Status not like '%CLOSE%'
and Status not like '%CANCEL%'
and [CleanType] like '%HIGH GRASS%'
AND [DateCreate] < '2020-02-12'
and ([DateFinish] >= '2020-02-12' or (Status not like '%CLOSE%' and  Status not like '%CANCEL%'))
''',conn2)

### Merging Datasets

CHIP/CSR

In [129]:
csr_chip = pd.merge(csr,chip,how='outer',left_on='srid',right_on='CSR_Num')

In [130]:
print(str(len(csr_chip[csr_chip['CB_ID'].isna()]))+ ' records exist in CSR but not in CHIP')

49 records exist in CSR but not in CHIP


In [131]:
print(str(len(csr_chip[csr_chip['srid'].isna()]))+ ' records exist in CHIP but not in CSR')

224 records exist in CHIP but not in CSR


In [132]:
csr_chip_mismatch = csr_chip.loc[csr_chip['srid'].isnull()==True,:].append(csr_chip[csr_chip['CB_ID'].isnull()==True])
csr_chip_mismatch.to_csv('csr_chip_mismatch.csv')

CHIP/Open Baltimore

In [134]:
chip_ob = pd.merge(chip,ob,how='outer',left_on='CSR_Num',right_on='servicerequestnum')

In [135]:
print(str(len(chip_ob[chip_ob['CB_ID'].isna()]))+ ' records exist in Open Baltimore but not in CHIP')

125 records exist in Open Baltimore but not in CHIP


In [136]:
print(str(len(chip_ob[chip_ob['servicerequestnum'].isna()]))+ ' records exist in CHIP but not in Open Baltimore')

228 records exist in CHIP but not in Open Baltimore


In [137]:
chip_ob_mismatch = chip_ob.loc[chip_ob['servicerequestnum'].isnull()==True,:].append(chip_ob[chip_ob['CB_ID'].isnull()==True])
chip_ob_mismatch.to_csv('chip_ob_mismatch.csv')

Open Baltimore/CSR

In [138]:
csr_ob = pd.merge(csr,ob,how='outer',left_on='srid',right_on='servicerequestnum')

In [139]:
print(str(len(csr_ob[csr_ob['servicerequestnum'].isna()]))+ ' records exist in CSR but not in Open Baltimore')

4 records exist in CSR but not in Open Baltimore


In [142]:
print(str(len(csr_ob[csr_ob['srid'].isna()]))+ ' records exist in Open Baltimore but not in CSR')

76 records exist in Open Baltimore but not in CSR


In [141]:
csr_ob_mismatch = csr_ob.loc[csr_ob['servicerequestnum'].isnull()==True,:].append(csr_ob[csr_ob['srid'].isnull()==True])
csr_ob_mismatch.to_csv('csr_ob_mismatch.csv')